In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import os
import pandas as pd
import numpy as np
import random
from ultralytics import YOLO
import mediapipe as mp
import torch
import torch.nn.functional as F
import cv2
import pickle
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch import nn, optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

----
파이널 액션 라벨만 영상 상단에 띄우기

In [ ]:
import cv2
import os
import pandas as pd
from collections import deque
import torch
import mediapipe as mp
import pickle
from ultralytics import YOLO

# LSTM 모델 정의
class SkeletonLSTM(nn.Module):
    def __init__(self, input_size=99, hidden_size=256, num_layers=2, num_classes=3, dropout_prob=0.3):
        super(SkeletonLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 모델 및 라벨 인코더 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = '/content/drive/MyDrive/test/best_lstm_behavior_model.pth'
label_encoder_path = '/content/drive/MyDrive/test/label_encoder.pkl'

model = SkeletonLSTM(input_size=99, hidden_size=256, num_layers=2, num_classes=3, dropout_prob=0.3).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

with open(label_encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)

# YOLO 및 Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

yolo_model = YOLO('yolov8s.pt')


input_video_dir = '/content/drive/MyDrive/DL/output_all/mediapipe/test_videos2'
output_video_dir = '/content/drive/MyDrive/DL/output_all/mediapipe/output_videos'
csv_output_dir = '/content/drive/MyDrive/DL/output_all/mediapipe/test_csvs2'


os.makedirs(output_video_dir, exist_ok=True)

sequence_length = 10
frame_queue = deque(maxlen=sequence_length)

def predict_final_label(csv_path):
    df = pd.read_csv(csv_path)
    landmarks_tensor = torch.tensor(df.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(landmarks_tensor)
        final_label_idx = torch.argmax(output, dim=1).item()
        final_label = label_encoder.inverse_transform([final_label_idx])[0]
    return final_label

def process_video(video_path, output_path, csv_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    final_label = predict_final_label(csv_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO로 사람 검출
        yolo_results = yolo_model(frame)

        for box in yolo_results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            label = int(box.cls[0].cpu().numpy())

            if label == 0:  # 'person' 클래스만 사용
                person_crop = frame[y1:y2, x1:x2]
                person_rgb = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
                results = pose.process(person_rgb)

                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        frame[y1:y2, x1:x2],
                        results.pose_landmarks,
                        mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                    )

                    landmarks = []
                    for landmark in results.pose_landmarks.landmark:
                        landmarks.extend([landmark.x, landmark.y, landmark.z])

                    frame_queue.append(landmarks)

                    if len(frame_queue) == sequence_length:
                        landmarks_tensor = torch.tensor(frame_queue, dtype=torch.float32).unsqueeze(0).to(device)
                        with torch.no_grad():
                            output = model(landmarks_tensor)
                            probabilities = torch.nn.functional.softmax(output, dim=1).cpu().numpy()[0]

                        # 파이널 액션과 동일한 라벨의 확률만 표시
                        final_label_idx = label_encoder.transform([final_label])[0]
                        final_prob = probabilities[final_label_idx]

                        label_text = f"{final_label}: {final_prob * 100:.2f}%"
                        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(frame, f"Final Action: {final_label}", (10, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")

# 비디오 파일 순회하며 처리
for video_file in os.listdir(input_video_dir):
    if video_file.endswith('.mp4'):
        video_path = os.path.join(input_video_dir, video_file)
        output_path = os.path.join(output_video_dir, f'processed_{video_file}')
        csv_path = os.path.join(csv_output_dir, f"{video_file}.csv")
        process_video(video_path, output_path, csv_path)

print("모든 비디오 파일 처리 완료!")

Output hidden; open in https://colab.research.google.com to view.